# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


In [108]:
from pathlib import Path

import pandas as pd
import xlwings as xw
from xlwings.constants import BordersIndex, BorderWeight, HAlign

In [109]:
DATA_DIR = Path('data/')
OUTPUT_DIR = Path('output/')

In [132]:
def format_header(cell):
    sheet.range(cell).api.Font.Bold = True
    sheet.range(cell).api.HorizontalAlignment = HAlign.xlHAlignCenter
    for col_idx in (
            BordersIndex.xlEdgeTop,
            BordersIndex.xlEdgeRight,
            BordersIndex.xlEdgeBottom,
            BordersIndex.xlEdgeLeft
    ):
        sheet.range(cell).api.Borders(col_idx).LineStyle = 1
        sheet.range(cell).api.Borders(col_idx).Weight = BorderWeight.xlThin

## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [110]:
reviews_df = pd.read_csv(
    DATA_DIR.joinpath('reviews_sample.csv'),
    sep=',',
    index_col=0,
    parse_dates=['date']
)
reviews_df.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [111]:
recipes_df = pd.read_csv(
    DATA_DIR.joinpath('recipes_sample_with_filled_nsteps.csv'),
    sep=',',
    usecols=['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'],
    parse_dates=['submitted']
)
recipes_df.head()

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [112]:
reviews_sample = reviews_df.sample(int(reviews_df.shape[0] * 0.05))
reviews_sample

,user_id,recipe_id,date,rating,review
680162,1574800,93426,2010-03-11,5,Pretty good when I made it. I definitely thoug...
392773,383346,149843,2007-12-26,5,It's delicious and I omitted the nutmeg. Than...
141109,28087,441475,2012-04-16,4,This is the second time I have attempted to ma...
531388,37636,290253,2008-04-27,4,When I made this recipe I came to a halt durin...
754073,177753,45008,2006-12-27,5,I decided to try this for breakfast this morni...
...,...,...,...,...,...
554752,142386,48907,2009-04-22,5,Very good. Thanks for posting.
355262,579911,92096,2010-01-10,5,"I was looking for a rich, thick, slightly swee..."
490863,47559,89834,2006-06-11,5,My chicken was not quite thawed when I started...
493043,340130,101891,2007-06-11,5,This was great and all we added was a bit of l...


In [113]:
recipes_sample = recipes_df.sample(int(recipes_df.shape[0] * 0.05))
recipes_sample

,name,id,minutes,submitted,description,n_ingredients
8018,crawfish fettuccini,76792,65,2003-11-17,"a friend from thibodaux, louisiana gave me thi...",13.0
1263,asian twist chicken salad,23567,15,2002-03-28,just a little different. if you like kick add ...,NaN
23583,savory lemon potatoes for two,75312,40,2003-11-03,this of coarse can de doubled!,5.0
4631,caramel apple streusel microwave,95747,17,2004-07-16,this is great to pop in the microwave while yo...,NaN
14366,ice wine infused peaches,179889,25,2006-07-31,"from the television show ""man-made food"" i wa...",4.0
...,...,...,...,...,...,...
12934,grilled mexican style corn,64853,20,2003-06-17,this is the best corn on the cob you will ever...,4.0
10230,easy squeeze honey butter,382927,10,2009-07-26,could anything be simpler than this? you might...,2.0
22888,rock salt roast,45030,140,2002-10-31,i'm told this is one of those recipes that loo...,4.0
810,apple and spice bread pudding,421824,55,2010-04-27,such a delicious old-fashioned dessert - every...,12.0


In [114]:
with pd.ExcelWriter(OUTPUT_DIR.joinpath('recipes.xlsx')) as writer:
    reviews_sample.to_excel(writer, sheet_name='Отзывы', index=False)
    recipes_sample.to_excel(writer, sheet_name='Рецепты', index=False)

IllegalCharacterError: 

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [137]:
wb = xw.Book(OUTPUT_DIR.joinpath('recipes.xlsx'))

In [138]:
sheet = wb.sheets['Рецепты']

In [139]:
seconds_assign = (recipes_sample['minutes'] * 60).rename('seconds_assign')
cell = (1, sheet.range('A1').expand('right').shape[1] + 1)
sheet.range(cell).options(index=False).value = seconds_assign
format_header(cell)

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [140]:
seconds_formula = '=C2 * 60'
col_idx = sheet.range('A1').expand('right').shape[1] + 1
sheet.range(1, col_idx).value = 'seconds_formula'
sheet.range((2, col_idx), (sheet.range('A1').expand('down').shape[0], col_idx)).formula = seconds_formula
format_header((1, col_idx))

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
formula = '=COUNTIF(B2=B2)'  # чем я занимаюсь -_- ?????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????

In [142]:
set(recipes_sample['id'].unique()) & set(reviews_sample['recipe_id'].unique())

{378,
 2519,
 4150,
 5418,
 8468,
 8962,
 10443,
 10869,
 11538,
 15012,
 15060,
 15389,
 15839,
 22125,
 22203,
 22390,
 22915,
 23705,
 23873,
 24094,
 24604,
 24616,
 24638,
 27081,
 28848,
 29251,
 31135,
 32973,
 33102,
 35287,
 37362,
 38249,
 42076,
 45977,
 47597,
 47666,
 48031,
 49200,
 49896,
 52035,
 54800,
 54875,
 55706,
 56864,
 58495,
 59386,
 61088,
 61755,
 64816,
 67570,
 68004,
 68063,
 68512,
 69768,
 72139,
 73219,
 78896,
 81319,
 82221,
 83061,
 83222,
 85336,
 90246,
 90516,
 91011,
 95747,
 95891,
 98950,
 103175,
 103732,
 104074,
 104270,
 107871,
 108302,
 108354,
 108605,
 110825,
 113385,
 115753,
 116887,
 117060,
 118743,
 122725,
 123415,
 123490,
 123503,
 123863,
 133815,
 135210,
 135339,
 135350,
 136548,
 138031,
 141906,
 142388,
 146325,
 154267,
 156169,
 156572,
 156925,
 158588,
 161373,
 162185,
 162661,
 165988,
 167341,
 168032,
 168785,
 172267,
 173742,
 174095,
 174503,
 175540,
 176199,
 179476,
 187306,
 188204,
 191866,
 195254,
 195

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом